In [1]:
import requests
from datetime import datetime, timedelta, timezone
from faker import Faker

base_url = 'http://localhost:1803'
api_base_url = f'{base_url}/api/v1'

aircrafts_url = f'{api_base_url}/aircrafts'
seats_url = f'{api_base_url}/seats'
flights_url = f'{api_base_url}/flights'
bookings_url = f'{api_base_url}/bookings'
users_url = f'{api_base_url}/users'
passengers_url = f'{api_base_url}/passengers'

s = requests.Session()
SEED = 18022004
fake = Faker()
Faker.seed(SEED)

In [2]:
def post_stuff(url, data):
    res = s.post(url, json = data)
    # print(res.text)

    return res.json()

def get_stuff(url):
    res = s.get(url)
    # print(res.text)

    return res.json()

def put_stuff(url, data):
    res = s.put(url, json = data)
    # print(res.text)

    return res.json()

def delete_stuff(url):
    res = s.delete(url)
    # print(res.text)

    return res.json()

In [3]:
def create_aircraft():
    data = {
        'aircraft_manufacturer': fake.company(),
        'aircraft_model': fake.word(),
        'aircraft_type': fake.word(),
        'capacity': 40,
    }

    return data

def create_flight():
    data = {
        'flight_number': f'QAL{fake.random_int(min = 100, max = 999)}',
        'departure_airport': 'SGN',
        'arrival_airport': 'HAN',
        'departure_time': '',
        'arrival_time': '',
        'board_time': '',
        'gate': fake.catch_phrase(),
        'price': 0,
        'status': 'upcoming',
        'aircraft_id': 1,
    }

    return data

In [4]:
post_stuff(aircrafts_url, create_aircraft())
post_stuff(aircrafts_url, create_aircraft())

{'aircraft_id': 2,
 'aircraft_manufacturer': 'Elliott-Fleming',
 'aircraft_type': 'free',
 'aircraft_name': '',
 'capacity': 40}

In [5]:
dep_flight = create_flight()
ret_flight = create_flight()
ret_flight['departure_airport'], ret_flight['arrival_airport'] = ret_flight['arrival_airport'], ret_flight['departure_airport']

tz = timezone(timedelta(hours = 7))
tz_hr = str(tz).strip('UTC')[:-3]
tz_min = str(tz).strip('UTC')[-2:]
year, month, day = 2024, 12, 22
ret_year, ret_month, ret_day = 2024, 12, 28

dep_flight['departure_time'] = datetime(year, month, day, 8, 0, 0, tzinfo = tz).isoformat()
dep_flight['arrival_time'] = datetime(year, month, day, 10, 0, 0, tzinfo = tz).isoformat()
dep_flight['board_time'] = datetime(year, month, day, 7, 20, 0, tzinfo = tz).isoformat()
dep_flight['price'] = fake.random_int(min = 10000, max = 20000) / 100

ret_flight['departure_time'] = datetime(ret_year, ret_month, ret_day, 15, 0, 0, tzinfo = tz).isoformat()
ret_flight['arrival_time'] = datetime(ret_year, ret_month, ret_day, 17, 0, 0, tzinfo = tz).isoformat()
ret_flight['board_time'] = datetime(ret_year, ret_month, ret_day, 14, 40, 0, tzinfo = tz).isoformat()
ret_flight['price'] = fake.random_int(min = 10000, max = 20000) / 100

In [6]:
dep_flights = []
for i in range(3):
    dep_flight['departure_time'] = datetime(year, month, day, 8 + i, 0, 0, tzinfo = tz).isoformat()
    dep_flight['arrival_time'] = datetime(year, month, day, 10 + i, 0, 0, tzinfo = tz).isoformat()
    dep_flight['board_time'] = datetime(year, month, day, 7 + i, 20, 0, tzinfo = tz).isoformat()
    dep_flight['price'] = fake.random_int(min = 10000, max = 20000) / 100
    resp = post_stuff(flights_url + '/withseats', dep_flight)

    dep_flights.append(resp)

ret_flights = []
for i in range(3):
    ret_flight['departure_time'] = datetime(ret_year, ret_month, ret_day, 15 + i, 0, 0, tzinfo = tz).isoformat()
    ret_flight['arrival_time'] = datetime(ret_year, ret_month, ret_day, 17 + i, 0, 0, tzinfo = tz).isoformat()
    ret_flight['board_time'] = datetime(ret_year, ret_month, ret_day, 14 + i, 40, 0, tzinfo = tz).isoformat()
    ret_flight['price'] = fake.random_int(min = 10000, max = 20000) / 100
    resp = post_stuff(flights_url + '/withseats', ret_flight)

    ret_flights.append(resp)

In [7]:
booking_date = datetime.now(tz).replace(microsecond = 0).isoformat()
booking = {
    'flight_id': 1,
    'user_id': 1,
    'passenger_id': 1,
    'seat_id': 1,
    'booking_time': booking_date,
    'status': 'upcoming',
}

booking['booking_time']

'2024-12-21T19:53:09+07:00'